# Descripcion del proyecto
La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del sprint de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.

Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

# Tabla de contenido
1. [Inicialización](#Inicialización)
2. [Informacion general](#Informacion-general)
3. [Segmetar los datos](#Segmetar-los-datos) 
4. [Calidad de los modelos](#Calidad-de-los-modelos) 

   4.1.[Arbol de decision](#Arbol-de-decision)
   
   4.2.[Bosque aleatorio](#Bosque-aleatorio)
   
   4.3.[Regresion logistica](#Regresion-logistica)
   
5. [Calidad del modelo en el conjunto de prueba](#Calidad-del-modelo-en-el-conjunto-de-prueba)
6. [Prueba de cordura al modelo](#Prueba-de-cordura-al-modelo)
7. [Conclusión general](#Conclusión-general)

## . Inicializacion 
Abre el archivo de datos y estudia la información general


In [1]:
# Cargar todas las librerías
import pandas as pd 
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, mean_squared_error

In [2]:
users_behavior = pd.read_csv('/datasets/users_behavior.csv',sep=',', header=0)

##  Informacion general del DataFrame 

In [3]:
## Imprime la información general/resumida sobre el DataFrame users_behavior
def first_lookup(datos):
    print('Primera filas:')
    print(datos.head())
    print()
    print('Informacion:')
    print(datos.info())
    print()
    print('El total de valores ausentes es:')
    print(datos.isna().sum())
    print()
    print('El total de valores duplicados es:')
    print(datos.duplicated().sum())
print(first_lookup(users_behavior))

Primera filas:
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

Informacion:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None

El total de valores ausentes es:
calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

El total de valores duplicados es:
0
None


In [4]:
users_behavior['messages'] = users_behavior['messages'].astype(int)
users_behavior['calls'] = users_behavior['calls'].astype(int)
users_behavior['gb_used'] = users_behavior['mb_used'] / 1024
users_behavior.drop('mb_used', axis=1, inplace=True)
print(users_behavior.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   is_ultra  3214 non-null   int64  
 4   gb_used   3214 non-null   float64
dtypes: float64(2), int64(3)
memory usage: 125.7 KB
None


In [5]:
#Preparar las caracteristicas y el objetivo
features = users_behavior.drop(['is_ultra'], axis=1)
target = users_behavior['is_ultra']

## .  Segmenta los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.

In [6]:
# Dividir los datos en conjuntos de entrenamiento (60%), validación (20%) y prueba (20%)
features_train, features_temp, target_train, target_temp =train_test_split(features, target, test_size= 0.4, random_state=54321) # 40% para validación y prueba)
features_valid, features_test, target_valid, target_test= train_test_split(features_temp, target_temp, test_size=0.5, random_state=54321 )# Divide el 40% en validación y prueba


## . Investiga la calidad de diferentes modelos cambiando los hiperparámetros. Describe brevemente los hallazgos del estudio.

### Arbol de decision

In [7]:
# Inicializar variables para almacenar los mejores resultados
best_accuracy = 0
best_depth= 0 
# Probar diferentres profundidades
for depth in range(1, 11):
    dt_model = DecisionTreeClassifier(
    max_depth=depth,
    criterion='gini',
    class_weight='balanced',
    random_state=54321
)
#Entrenar el modelo
    dt_model.fit(features_train, target_train)
#Realizar las predicciones del conjunto de validacion
    dt_predictions = dt_model.predict(features_valid)
    dt_accuracy = accuracy_score(target_valid, dt_predictions)
    print(f"Árbol de Decisión - max_depth: {depth} - Exactitud: {dt_accuracy:.2f}")
#Actualizar los mejores parametros si se encuentra una mejor exactitud
    if dt_accuracy > best_accuracy:
        best_accuracy = dt_accuracy
        best_depth = depth
#imprimir los mejores parametros encontrados
print('Mejor profundidad encontrada:', best_depth)
print('Mejor exactitud:', best_accuracy)


Árbol de Decisión - max_depth: 1 - Exactitud: 0.71
Árbol de Decisión - max_depth: 2 - Exactitud: 0.74
Árbol de Decisión - max_depth: 3 - Exactitud: 0.76
Árbol de Decisión - max_depth: 4 - Exactitud: 0.73
Árbol de Decisión - max_depth: 5 - Exactitud: 0.76
Árbol de Decisión - max_depth: 6 - Exactitud: 0.76
Árbol de Decisión - max_depth: 7 - Exactitud: 0.76
Árbol de Decisión - max_depth: 8 - Exactitud: 0.75
Árbol de Decisión - max_depth: 9 - Exactitud: 0.72
Árbol de Decisión - max_depth: 10 - Exactitud: 0.72
Mejor profundidad encontrada: 7
Mejor exactitud: 0.76049766718507


El resultado de exactitud para mi modelo de Arbol de decision supera el umbral de 0.75 que se ha establecido, modelo predijo correctamente el plan Smart o ultra en un 76.05% de los casos del conjunto de validacion con una profundidad de 7 logrando un mejor equilibrio entre la complejidad del modelo y su capacidad para generalizar nuevos datos. A medida que aumentas la profundidad del arbol alzanzando su punto maximo en max_depth= 3, pero en el punto tiene a estabilizarse y tambien a disminuir lo cual indica un sobreajusto a profunidades mas altas, el uso de class_weight='balanced ha ajustado el modelo para majear posibles desabalances en las clases

### Bosque aleatorio

In [8]:
# Se inicia y se entrena el modelo de Random Forest  implementando hiperparametros
best_accuracy= 0
best_params ={}
#Probar diferentes combinaciones de hiperparametros
for est in range(10, 51, 10): # n_estimators desde 10 hasta 50 (incrementos de 10)
    for depth in range (1, 11): # max_depth desde 1 hasta 10
        model= RandomForestClassifier(random_state=54321, n_estimators=est, max_depth=depth)
        #Entrenar el modelo
        model.fit(features_train, target_train)
        # Realizar predicciones en el conjunto de validacion
        rf_predictions = model.predict(features_valid)
        #Calcular la exactitud del modelo
        rf_accuracy= accuracy_score(target_valid, rf_predictions)
        print(f"Random Forest - n_estimators: {est}, max_depth: {depth} - Exactitud: {rf_accuracy:.2f}")
        # Actualizar los mejores parámetros si se encuentra una mejor exactitud
        if rf_accuracy > best_accuracy:
            best_accuracy = rf_accuracy
            best_params = {'n_estimators': est, 'max_depth': depth}
#Imprimir los mejores parametros encontrados
print('Mejores parametros encontrados:')
print(best_params)
print('Mejor exactitud:', best_accuracy)
            
        

Random Forest - n_estimators: 10, max_depth: 1 - Exactitud: 0.72
Random Forest - n_estimators: 10, max_depth: 2 - Exactitud: 0.75
Random Forest - n_estimators: 10, max_depth: 3 - Exactitud: 0.75
Random Forest - n_estimators: 10, max_depth: 4 - Exactitud: 0.79
Random Forest - n_estimators: 10, max_depth: 5 - Exactitud: 0.78
Random Forest - n_estimators: 10, max_depth: 6 - Exactitud: 0.78
Random Forest - n_estimators: 10, max_depth: 7 - Exactitud: 0.79
Random Forest - n_estimators: 10, max_depth: 8 - Exactitud: 0.80
Random Forest - n_estimators: 10, max_depth: 9 - Exactitud: 0.79
Random Forest - n_estimators: 10, max_depth: 10 - Exactitud: 0.80
Random Forest - n_estimators: 20, max_depth: 1 - Exactitud: 0.70
Random Forest - n_estimators: 20, max_depth: 2 - Exactitud: 0.73
Random Forest - n_estimators: 20, max_depth: 3 - Exactitud: 0.77
Random Forest - n_estimators: 20, max_depth: 4 - Exactitud: 0.78
Random Forest - n_estimators: 20, max_depth: 5 - Exactitud: 0.78
Random Forest - n_estima

Para los mejores hiperparametros encontrados n_estimartors de 10, max_depth de 10 con una exactitud de 0.8025 el modelo Random Forest ha logrado superar el umbral objetivo de 0.75, se observa que la exactitud aumenta conforme aummenta la profundidad hasta cierto punto y se mantiene relativamente constante con diferentes valores de n_estimators, mostrando que al aumentar el numero de arboles no siempre mejora la precision

### Regresion logistica

In [9]:
# Inicializar el modelo de Regresion logistica
logistic_model = LogisticRegression(solver='liblinear', random_state=54321)
#Entrenar el modelo
logistic_model.fit(features_train, target_train)
#Realizar predicciones y evaluar el modelo
#Realizar predicciones del conjunto de validacion
lr_predictions = logistic_model.predict(features_valid)
#Calcular la exactitud del modelo
lr_accuracy= accuracy_score(target_valid, lr_predictions)
print('Regresion logistica - Exactitud:', lr_accuracy)

Regresion logistica - Exactitud: 0.7060653188180405


La exactitud obtenida mediante el modelo de regresion lostica obtuvo 0.7061 lo cual no logro superar el umbral propuesto de 0.75 este modelo no se consideraria como opcion a eleigir por esta razon y porque el modelo random forest y decision tree tuvieron una mejor exaxctitud. 

## . Comprueba la calidad del modelo usando el conjunto de prueba.

In [11]:
# Inicializar el modelo final con los mejores parametros encontrados
final_rf_model = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=54321)
#Entrenar el modelo en el conjunto de entrenamiento
final_rf_model.fit(features_train, target_train)
#Realizar predicciones en el conjunto de prueba
test_predictions_rf = final_rf_model.predict(features_test)
#Calcular la aecactitud en el conjunto de prueba
test_accuracy = accuracy_score(target_test, test_predictions_rf)
print('Exactitud del modelo Random Forest en el conjunto de prueba:', test_accuracy)
rmse = mean_squared_error(target_test, test_predictions_rf, squared=False)
print('rmse:', rmse)


Exactitud del modelo Random Forest en el conjunto de prueba: 0.8242612752721618
rmse: 0.4192120283673147


La exactitud del modelo Random Forest en el conjunto de validacion fue de 0.8025 superior en comparacion con Decision Tree y con la Regresion Logistica y tambien superior con la exactitud del modelo Random Forest en el conjunto de prueba de 0.8243  sugiere que es mas efectivo para capturar las relaciones en los datos, puede deberse a su capacidad para manejar  interacciones complejas entre caracteristicas y su robustez frente al sobreajuste al utilizar multiples arboles, el modelo puede manejar mejor la variablidad  y el ruido de los datos

## . Tarea adicional: haz una prueba de cordura al modelo. Estos datos son más complejos que los que habías usado antes así que no será una tarea fácil. Más adelante lo veremos con más detalle.

In [16]:
# Calcular el valor promedio  para los elementos del conjunto de prueba del target 
predictions_avg = pd.Series(target_test.mean(), index=target_test.index)# Una vez calcula el promedio del target_test se convierte a un series
rmse = mean_squared_error(target_test, predictions_avg, squared=False) # Calcular el rmse que proviene de nuestro mse
print('rmse:', rmse)


rmse: 0.4466513167022035


Para el analisis de mi prueba de cordura
a) El calculo del promedio target de mi conjunto de prueba y se ha convertido en una serie devolviendo las predicciones del promedio donde aparece el plan ultra
b) Para el calculo del rmse del error cuadratico medio entre el target_test y predictions_avg el valor obtenido se compara con el rmse del modelo de Random forest 
c) Comparacion rmse del valor promedio del target del conjunto de prueba fue de 0.4467 fue mayor que el rmse del modelo de Random forest del cojunto de prueba fue de 0.4193 al tenero un valor mas bajo de nuestro modelo es correcto se acerca mas a los valores reales un valor bajo de rmse, mi prueba de cordura usando rmse me proporciona infomacion util para la calidad de mi modelo

## . Conclusion general

El proposito del proyecto La compañía móvil Megaline quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra. Donde al segmenta los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba se realizo los siguiente con la meta de que el umbral de exactitud es 0.75
a) Se analizo la calidad de diferentes modelos cambiando los hiperparámetros primer modelo el arbol de decision con diferentes profunidas para observar cual presenta mayor exactitud sus resultados fueron (mejor profundidad encontrada: 7
, mejor exactitud: 0.76049766718507) a simple vista dirias cumplio el propisito del proyecto, pero se el segundo modelo a analizar fue el bosque aleatorio con los resultados obtenidos (Mejores parametros encontrados:{'n_estimators': 10, 'max_depth': 10}Mejor exactitud: 0.80248833592535) donde sus hiperparametros el numero de arboles, profundidad se analizaron con diferentes valores para no establecer arbitrariamente un valor sino que establecer valores y observar como se comportaba el modelo y obtuvo una mejor exactitud que el arbol de decision. Por ultimo la regresion logistica que tuvo un modelo con los siguientes resultados: Regresion logistica - Exactitud: 0.7060653188180405 fue de los tres modelos la mas baja este modelo no nos funciona, el ideal es el bosque aleatorio al aumentar el numero de arboles aumenta la exactitud.
b) Comprueba la calidad del modelo usando el conjunto de prueba del modelo de bosque aleatorio Exactitud del modelo Random Forest en el conjunto de prueba: 0.8242612752721618 presento una mejoria en cuanto a su exactitud esto confirma que seria el modelo a elegir pero aun queda la prueba de cordura.
c) El analsis de la prueba de cordura al obtener el promedio de target_test  representa si un cliente esta en el plan Ultra y se obtuvo un 27.53%, el calculo de rmse de 0.4467  que representa la diferencia promedio entre las predicciones basadas en el promedio y los valores reales y en comparacion con el rmse del modelo random forest  de 0.4192  esto indica que el modelo random forest  realiza un mejor trabajo al predecir los valores reales y un valor bajo de rmse nos indica un mejor ajuste del modelo a los datos. Se entreno un modelo utilizando datos sobre el comportamiento de los clientes que ya se han cambiado a los planes nuevos donde se incluye: número de llamadas,duración total de la llamada, numero de mensajes de texto, trafico de internet, la variable objetivo es is_ultra, que indica si un cliente está en el plan Ultra (1) o Smart (0). Al obtner la mejor exactitud para el modelo random forest esto indica que el modelo es capaz de predecir correctamente que clientes deberian estar en el plan Ultra, que actualmente están en planes heredados podrían beneficiarse al cambiarse al plan Ultra.

